# Streaming

This notebook covers functionality related to streaming.

For more information, see:

* [Streaming with LCEL](https://python.langchain.com/docs/expression_language/interface#stream)
* [Streaming for RAG](https://python.langchain.com/docs/use_cases/question_answering/streaming)
* [Streaming for Agents](https://python.langchain.com/docs/modules/agents/how_to/streaming)

## Basic Streaming

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain = prompt | model | output_parser

In [3]:
for s in chain.stream({"topic": "bears"}):
    print(s, end="")

Sure, here's a bear joke for you:

Why do bears never get asked to play cards?

Because they're always caught "bear-handed"!

## Streaming with RunnableParallel

In [4]:
from langchain_core.runnables import RunnableParallel

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain1 = prompt | model | output_parser

prompt = ChatPromptTemplate.from_template("Write me a poem about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain2 = prompt | model | output_parser

parallel_chain = RunnableParallel({
    "joke": chain1,
    "poem": chain2
})

for s in parallel_chain.stream({"topic": "bears"}):
    print(s)

{'poem': ''}
{'poem': 'In'}
{'poem': ' forests'}
{'poem': ' deep'}
{'poem': ','}
{'poem': ' where'}
{'poem': ' shadows'}
{'joke': ''}
{'poem': ' dwell'}
{'poem': ',\n'}
{'poem': 'Beh'}
{'poem': 'old'}
{'poem': ' the'}
{'poem': ' creatures'}
{'joke': 'Sure'}
{'joke': ','}
{'joke': ' here'}
{'poem': ','}
{'poem': ' fierce'}
{'poem': ' and'}
{'joke': "'s"}
{'joke': ' a'}
{'poem': ' fell'}
{'joke': ' bear'}
{'poem': ',\n'}
{'poem': 'With'}
{'joke': ' joke'}
{'joke': ' for'}
{'joke': ' you'}
{'joke': ':\n\n'}
{'joke': 'Why'}
{'joke': ' don'}
{'joke': "'t"}
{'joke': ' bears'}
{'joke': ' wear'}
{'poem': ' strength'}
{'poem': ' unmatched'}
{'poem': ','}
{'poem': ' and'}
{'poem': ' claws'}
{'joke': ' shoes'}
{'joke': '?\n\n'}
{'joke': 'Because'}
{'poem': ' so'}
{'poem': ' sharp'}
{'poem': ',\n'}
{'joke': ' they'}
{'joke': ' have'}
{'joke': ' bear'}
{'joke': ' feet'}
{'joke': '!'}
{'joke': ''}
{'poem': 'The'}
{'poem': ' mighty'}
{'poem': ' bears'}
{'poem': ','}
{'poem': ' the'}
{'poem': ' rulers

In [5]:
for s in parallel_chain.invoke({"topic": "bears"}):
    print(s)

joke
poem


In [6]:
repr(s)

"'poem'"

## Stream Log

In [7]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever
from langchain_core.runnables import RunnablePassthrough

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

chain = prompt | model | output_parser

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever.with_config(run_name="Docs")
) | chain

In [8]:
for s in retrieval_chain.stream({"question": "what is langsmith"}):
    print(s, end="")

LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It provides tools to trace and evaluate language model applications and intelligent agents, as well as to track system-level and model/chain performance. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs.

In [9]:
async for s in retrieval_chain.astream_log({"question": "what is langsmith"}):
    print(s, end="")

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '21100807-4323-43e2-9e62-28220533198b',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})RunLogPatch({'op': 'add',
  'path': '/logs/RunnableAssign<context>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '19efbacc-1f3d-41f3-a2be-3af8fc55a469',
            'metadata': {},
            'name': 'RunnableAssign<context>',
            'start_time': '2024-01-29T09:34:46.078+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})RunLogPatch({'op': 'add',
  'path': '/logs/RunnableAssign<context>/streamed_output/-',
  'value': {'question': 'what is langsmith'}})RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel<context>',
  'value': {'end_time': None,
            'final_output': None,
  

In [10]:
async for s in retrieval_chain.astream_log({"question": "what is langsmith"}, include_names=["Docs"]):
    print(s, end="")

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '37be2b95-3f5e-40e4-bf46-7db54659028e',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '1bfa4c12-c2b7-4f55-9a5c-676f23699bcc',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-01-29T09:36:37.861+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:2'],
            'type': 'retriever'}})RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='LangSmith | 🦜️🔗 Langchain LangSmith LangSmith LangSmith helps you trace and evaluate your language model applications and intelligent agents to help you move from prototype to production. Chec

## Agents
# Stream Actions

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [search]

# Get the prompt to use - you can modify this!
# If you want to see the prompt in full, you can at: https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [12]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk)
    print("------")

{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})])], 'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]}
------
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]), 

## Stream Tokens

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [14]:
async for chunk in agent_executor.astream_log(
    {"input": "what is the weather in sf", "chat_history": []},
    include_names=["ChatOpenAI"],
):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '33d03f5d-bf44-4e12-b3e5-61b0b99b3ef3',
            'logs': {},
            'name': 'AgentExecutor',
            'streamed_output': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI',
  'value': {'end_time': None,
            'final_output': None,
            'id': '4e06fbc3-bee4-49f8-a88f-f26411827538',
            'metadata': {},
            'name': 'ChatOpenAI',
            'start_time': '2024-01-29T09:52:29.203+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:3'],
            'type': 'llm'}})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '', 'name': 'tavily_search_results_json'}})})
RunLogPatch(